In [ ]:
%%writefile /kaggle/working/r4.2/optimized_feature_extraction.py

#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Optimized Feature Extraction v3.5 (Fix: Ambiguous Truth Value in Email Logic)
"""
import os, sys, time
import pandas as pd
import numpy as np
from datetime import datetime
import shutil
from joblib import Parallel, delayed
import glob

# --- CẤU HÌNH ---
DNAME = 'r4.2'
NUM_WEEK = 73 
CHUNK_SIZE = 5000000 

# MASTER SCHEMA (Để chuẩn hóa cột CSV)
ALL_COLS = [
    'id', 'date', 'user', 'pc', 'activity', 'type', 
    'to', 'cc', 'bcc', 'from', 'size', '#att', 'content', 
    'url/fname'
]

# ==========================================
# 1. HELPER & VECTORIZED LOGIC
# ==========================================
def extract_row_features_vectorized(df, users_df):
    # 1. TIME
    df['hour'] = df['dt'].dt.hour
    df['weekday'] = df['dt'].dt.weekday
    is_weekend = df['weekday'] >= 5
    is_after = (df['hour'] < 8) | (df['hour'] > 17)
    conditions = [ (is_weekend & is_after), is_weekend, is_after ]
    choices = [4, 3, 2]
    df['time'] = np.select(conditions, choices, default=1)
    
    # 2. PC CHECK
    df = df.merge(users_df[['pc']], left_on='user', right_index=True, how='left')
    df['pc_type'] = np.where(df['pc_x'] == df['pc_y'], 0, 2)
    df.drop(columns=['pc_y'], inplace=True)
    df.rename(columns={'pc_x': 'pc'}, inplace=True)

    # 3. FEATURES
    feature_cols = [f'f{i}' for i in range(20)]
    for c in feature_cols: df[c] = 0
    
    # FILE
    is_file = df['type'] == 'file'
    if is_file.any():
        fname_vals = df.loc[is_file, 'url/fname'].astype(str).fillna('').values
        content_vals = df.loc[is_file, 'content'].astype(str).fillna('').values
        
        ftype = np.ones(len(fname_vals))
        fname_s = pd.Series(fname_vals)
        ftype[fname_s.str.contains(r'\.zip|\.rar|\.7z', regex=True)] = 2
        ftype[fname_s.str.contains(r'\.jpg|\.png|\.bmp', regex=True)] = 3
        ftype[fname_s.str.contains(r'\.doc|\.pdf', regex=True)] = 4
        ftype[fname_s.str.contains(r'\.txt|\.cfg', regex=True)] = 5
        ftype[fname_s.str.contains(r'\.exe|\.sh', regex=True)] = 6
        
        df.loc[is_file, 'f1'] = ftype
        df.loc[is_file, 'f2'] = [len(s) for s in content_vals]
        df.loc[is_file, 'f3'] = [s.count(' ')+1 for s in content_vals]
        df.loc[is_file, 'f4'] = np.where(pd.Series(fname_vals).str.startswith('C'), 1, 
                                np.where(pd.Series(fname_vals).str.startswith('R'), 2, 0))
        df.loc[is_file, 'f5'] = [s.count('\\') for s in fname_vals]

    # HTTP
    is_http = df['type'] == 'http'
    if is_http.any():
        url_vals = df.loc[is_http, 'url/fname'].astype(str).fillna('').values
        content_vals = df.loc[is_http, 'content'].astype(str).fillna('').values
        
        htype = np.ones(len(url_vals))
        url_s = pd.Series(url_vals)
        htype[url_s.str.contains('facebook|twitter|instagram', case=False)] = 2
        htype[url_s.str.contains('dropbox|drive.google', case=False)] = 3
        htype[url_s.str.contains('job|indeed|recruit', case=False)] = 4
        htype[url_s.str.contains('wikileaks|hack', case=False)] = 5
        
        df.loc[is_http, 'f6'] = htype
        df.loc[is_http, 'f7'] = [len(s) for s in url_vals]
        df.loc[is_http, 'f8'] = [s.count('/')-2 for s in url_vals]
        df.loc[is_http, 'f9'] = [len(s) for s in content_vals]
        df.loc[is_http, 'f10'] = [s.count(' ')+1 for s in content_vals]

    # EMAIL
    is_email = df['type'] == 'email'
    if is_email.any():
        to_vals = df.loc[is_email, 'to'].astype(str).fillna('').values
        bcc_vals = df.loc[is_email, 'bcc'].astype(str).fillna('').values
        content_vals = df.loc[is_email, 'content'].astype(str).fillna('').values
        
        n_to = [s.count(';')+1 if s else 0 for s in to_vals]
        n_bcc = [s.count(';')+1 if s else 0 for s in bcc_vals]
        
        df.loc[is_email, 'f11'] = np.array(n_to) + np.array(n_bcc)
        df.loc[is_email, 'f12'] = df.loc[is_email, '#att'].fillna(0)
        
        to_s = pd.Series(to_vals)
        bcc_s = pd.Series(bcc_vals)
        
        # --- FIX LỖI TẠI ĐÂY: Thay 'or' bằng '|' và đóng ngoặc ---
        # Logic: (To nội bộ) VÀ (Bcc rỗng HOẶC Bcc nội bộ) -> Internal (0)
        # Ngược lại -> External (1)
        # Lưu ý: 'dtaa.com' là domain công ty trong dataset CERT
        
        is_internal_to = to_s.str.contains('dtaa.com', regex=False)
        is_internal_bcc = (bcc_s == '') | bcc_s.str.contains('dtaa.com', regex=False)
        
        # Nếu cả To và Bcc đều an toàn -> 0, ngược lại -> 1
        xemail = np.where(is_internal_to & is_internal_bcc, 0, 1)
        
        df.loc[is_email, 'f13'] = xemail
        df.loc[is_email, 'f17'] = df.loc[is_email, 'size'].fillna(0)
        df.loc[is_email, 'f18'] = [len(s) for s in content_vals]
        df.loc[is_email, 'f19'] = [s.count(' ')+1 for s in content_vals]

    # Act Code
    conditions = [
        df['type'] == 'logon', df['type'] == 'device', 
        df['type'] == 'http', df['type'] == 'email', df['type'] == 'file'
    ]
    choices = [1, 3, 5, 6, 7]
    df['act_code'] = np.select(conditions, choices, default=0)
    
    df.loc[(df['type']=='logon') & (df['activity']!='Logon'), 'act_code'] = 2
    df.loc[(df['type']=='device') & (df['activity']!='Connect'), 'act_code'] = 4
    
    return df

# ==========================================
# 2. STEP 1: FAST SPLIT (Standardized)
# ==========================================
def fast_split_by_week():
    # Kiểm tra số lượng file CSV
    if os.path.exists("DataByWeek") and len(glob.glob("DataByWeek/*.csv")) > 60:
        print("⚡ Step 1 (Split) đã có dữ liệu hợp lệ. Bỏ qua.")
        return

    print("🚀 Step 1: Tách dữ liệu (Vectorized + Standardized)...")
    if os.path.exists("DataByWeek"): shutil.rmtree("DataByWeek")
    os.makedirs("DataByWeek")

    START_DATE = datetime.strptime('2010-01-02', '%Y-%m-%d')
    files = ['device.csv', 'email.csv', 'file.csv', 'http.csv', 'logon.csv']
    
    for fname in files:
        if not os.path.exists(fname): continue
        print(f"  -> Xử lý {fname}...")
        act_type = fname.split('.')[0]
        
        for chunk in pd.read_csv(fname, chunksize=CHUNK_SIZE, on_bad_lines='skip', engine='python'):
            chunk['dt'] = pd.to_datetime(chunk['date'], format="%m/%d/%Y %H:%M:%S")
            chunk['week_num'] = (chunk['dt'] - START_DATE).dt.days // 7
            chunk['type'] = act_type
            
            # CHUẨN HÓA CỘT
            chunk = chunk.reindex(columns=ALL_COLS + ['week_num', 'dt'])
            
            for w, group in chunk.groupby('week_num'):
                if w < 0 or w >= NUM_WEEK: continue
                save_path = f"DataByWeek/{int(w)}.csv"
                header = not os.path.exists(save_path)
                group.drop(columns=['week_num', 'dt']).to_csv(save_path, mode='a', index=False, header=header)

# ==========================================
# 3. STEP 2: USER & LABEL
# ==========================================
def get_mal_userdata():
    possible_paths = ["/kaggle/input/cert42-answer/answers"]
    found = False
    for p in possible_paths:
        if os.path.exists(os.path.join(p, "insiders.csv")):
            if not os.path.exists("answers"):
                try: os.symlink(p, "answers")
                except: shutil.copytree(p, "answers")
            found = True
            break
            
    print("  -> Đang đọc LDAP...")
    allfiles = glob.glob('LDAP/*.csv')
    alluser = {}
    for file in sorted(allfiles):
        month = os.path.basename(file).split('.')[0]
        df_ldap = pd.read_csv(file)
        for _, row in df_ldap.iterrows():
            if row['user_id'] not in alluser:
                alluser[row['user_id']] = row.drop('user_id').tolist() + [month, np.nan]

    if os.path.isfile("psychometric.csv"):
        p_score = pd.read_csv("psychometric.csv")
        for _, row in p_score.iterrows():
            if row['user_id'] in alluser: 
                alluser[row['user_id']] += row.iloc[2:].tolist()

    cols = ['uname', 'email', 'role', 'b_unit', 'f_unit', 'dept', 'team', 'sup', 'wstart', 'wend', 'O', 'C', 'E', 'A', 'N']
    df = pd.DataFrame.from_dict(alluser, orient='index', columns=cols)
    
    if os.path.exists("DataByWeek/1.csv"):
        try:
            w1 = pd.read_csv("DataByWeek/1.csv", usecols=['user','pc'])
            user_pcs = w1.groupby('user')['pc'].apply(lambda x: list(set(x))).to_dict()
        except: user_pcs = {}
    else: user_pcs = {}
    df['pc'] = df.index.map(lambda x: user_pcs.get(x, [None])[0])

    print("  -> Nạp danh sách hành vi Insider...")
    malicious_ids_set = set()
    if os.path.exists("answers/insiders.csv"):
        insiders = pd.read_csv("answers/insiders.csv")
        insiders = insiders[insiders['dataset'].astype(str) == '4.2']
        for _, row in insiders.iterrows():
            path1 = f"answers/{row['details']}"
            path2 = f"answers/r4.2-{row['scenario']}/{os.path.basename(row['details'])}"
            final = path1 if os.path.exists(path1) else (path2 if os.path.exists(path2) else None)
            if final:
                try:
                    mal_log = pd.read_csv(final, header=None, engine='python', on_bad_lines='skip')
                    if mal_log.shape[1] > 1: malicious_ids_set.update(mal_log[1].values)
                except: pass
    
    print(f"     => {len(malicious_ids_set)} hành vi độc hại.")
    return df, malicious_ids_set

# ==========================================
# 4. STEP 3: PIPELINE (PARQUET)
# ==========================================
def process_week_vectorized(week, users_df, mal_ids_set):
    csv_path = f"DataByWeek/{week}.csv"
    if not os.path.exists(csv_path): return
    try: df = pd.read_csv(csv_path, low_memory=False)
    except: return
    if df.empty: return
    
    df['dt'] = pd.to_datetime(df['date'], format="%m/%d/%Y %H:%M:%S")
    df.sort_values('dt', inplace=True)
    
    # Extract Features
    df = extract_row_features_vectorized(df, users_df)
    
    # Label
    df['mal_act'] = df['id'].isin(mal_ids_set).astype(int)
    
    week_res = []
    
    for u, u_group in df.groupby('user'):
        if u not in users_df.index: continue
        
        u_static = users_df.loc[u, ['O','C','E','A','N','role','dept']].tolist()
        dyn_feats = [1 if (u_group['time'] >= 3).any() else 0] 
        
        act_col_map = {
            3: ['f0'], 7: ['f1','f2','f3','f4','f5'],
            6: ['f11','f12','f17','f19'], 5: ['f7','f8','f10']
        }
        
        for act_code in [1, 3, 5, 6, 7]:
            cols_to_stat = act_col_map.get(act_code, [])
            act_rows = u_group[u_group['act_code'] == act_code]
            
            for tm in ['all', 1, 2, 3]:
                if tm == 'all': curr = act_rows
                elif tm == 3: curr = act_rows[act_rows['time'] >= 3]
                else: curr = act_rows[act_rows['time'] == tm]
                
                stats = [len(curr)]
                for c in cols_to_stat:
                    if len(curr) > 0:
                        vals = pd.to_numeric(curr[c], errors='coerce').values
                        stats.extend([np.nanmean(vals), np.nanstd(vals), np.nanmin(vals), np.nanmax(vals)])
                    else: stats.extend([0,0,0,0])
                dyn_feats.extend(stats)

        is_insider = 1 if u_group['mal_act'].sum() > 0 else 0
        week_res.append([week, u] + u_static + dyn_feats + [is_insider])

    if week_res:
        out_dir = 'ExtractedData/week_parquet'
        os.makedirs(out_dir, exist_ok=True)
        res_df = pd.DataFrame(week_res)
        res_df.columns = res_df.columns.astype(str)
        res_df.to_parquet(f"{out_dir}/part_{week}.parquet", index=False)

if __name__ == "__main__":
    st = time.time()
    
    fast_split_by_week()
    print(f"✅ Step 1 Done. Time: {(time.time()-st)/60:.2f} m")
    
    users, mal_ids = get_mal_userdata()
    print(f"✅ Step 2 Done. Time: {(time.time()-st)/60:.2f} m")
    
    print("🚀 Step 3: Tính toán & Xuất Parquet...")
    Parallel(n_jobs=4)(delayed(process_week_vectorized)(i, users, mal_ids) for i in range(NUM_WEEK))
    print(f"🎉 ALL DONE. Total Time: {(time.time()-st)/60:.2f} m")

    # Kiểm tra lần cuối
    pq_files = glob.glob("ExtractedData/week_parquet/*.parquet")
    if len(pq_files) > 0:
        print(f"-> THÀNH CÔNG: Tìm thấy {len(pq_files)} file parquet.")
    else:
        print("-> CẢNH BÁO: Vẫn không thấy file parquet. Kiểm tra lại Logs.")

In [ ]:
# Xóa thư mục cũ chứa file lỗi
!rm -rf /kaggle/working/act_type_map.json
!rm -rf /kaggle/working/dept_map.json
!rm -rf /kaggle/working/role_map.json
!rm -rf /kaggle/working/role_map.json
!rm -rf /kaggle/working/team_map.json
!rm -rf /kaggle/working/tmp_device
!rm -rf /kaggle/working/tmp_email
!rm -rf /kaggle/working/tmp_file
!rm -rf /kaggle/working/tmp_http
!rm -rf /kaggle/working/tmp_logon

print("Đã dọn dẹp file lỗi. Bây giờ bạn hãy chạy lại script chính nhé!")

In [ ]:
# ================= CERT r4.2 ACTION-LEVEL MULTICLASS UEBA PIPELINE =================
# One-cell | Kaggle-ready | Deep Learning + Explainable

import os, shutil, time, json
import pandas as pd
import numpy as np
from joblib import Parallel, delayed

# ================= CONFIG =================
SOURCE_DIR = '/kaggle/input/cert-r4-2/archive'
ANSWERS_DIR = '/kaggle/input/cert42-answer/answers'
OUTPUT_DIR = '/kaggle/working'
CHUNK_SIZE = 2_000_000
NUM_CORES = 4

ACT_MAP = {'logon':1, 'device':2, 'file':3, 'email':4, 'http':5}
#ORG_DOMAIN = 'dtaa.com'
#EXEC_EXT = ('.exe','.zip','.tar')
#SENSITIVE_PREFIX = 'R'
#RISKY_HTTP = ['job','hack','leak','upload','drive','dropbox','keylog']

# ================= 1. USER CONTEXT =================
def load_user_context():
    ldap_dir = os.path.join(SOURCE_DIR,'LDAP')
    if not os.path.exists(ldap_dir): ldap_dir = SOURCE_DIR

    users={}
    for f in os.listdir(ldap_dir):
        if f.endswith('.csv'):
            df=pd.read_csv(os.path.join(ldap_dir,f))
            for _,r in df.iterrows():
                users[r['user_id']] = r.to_dict()

    df_users=pd.DataFrame.from_dict(users,orient='index')

    psy=os.path.join(SOURCE_DIR,'psychometric.csv')
    if os.path.exists(psy):
        df_users=df_users.merge(pd.read_csv(psy),on='user_id',how='left')
    else:
        for c in ['O','C','E','A','N']: df_users[c]=0.5

    def enc(col): return {v:i+1 for i,v in enumerate(sorted(col.fillna('UNK').unique()))}

    role_map,dept_map,team_map=enc(df_users['role']),enc(df_users['department']),enc(df_users['team'])

    for m,n in [(role_map,'role'),(dept_map,'dept'),(team_map,'team')]:
        with open(f'{OUTPUT_DIR}/{n}_map.json','w') as f:
            json.dump({str(v):k for k,v in m.items()},f,indent=2)

    with open(f'{OUTPUT_DIR}/act_type_map.json','w') as f:
        json.dump({str(v):k for k,v in ACT_MAP.items()},f,indent=2)

    ctx={}
    for _,r in df_users.iterrows():
        ctx[r['user_id']]=[
            role_map.get(r['role'],0),
            dept_map.get(r['department'],0),
            team_map.get(r['team'],0),
            float(r['O']),float(r['C']),float(r['E']),float(r['A']),float(r['N'])
        ]
    return ctx

# ================= 2. LOAD MALICIOUS IDS =================
def load_malicious_ids():
    df=pd.read_csv(os.path.join(ANSWERS_DIR,'insiders.csv'))
    df=df[df['dataset'].astype(str)=='4.2']
    ids=set()
    for _,r in df.iterrows():
        f=os.path.join(ANSWERS_DIR,f"r4.2-{r['scenario']}",r['details'])
        if os.path.exists(f):
            with open(f) as fh:
                for l in fh:
                    ids.add(l.strip().split(',')[1].replace('{','').replace('}',''))
    return mal_ids

# ================= 3. FEATURE ENGINEERING =================
# def add_time_features(df):
#     df['dt']=pd.to_datetime(df['date'],errors='coerce')
#     df=df.dropna(subset=['dt'])
#     df['ts']=(df['dt'].astype('int64')//10**9).astype(np.int64)
#     df['hour']=df['dt'].dt.hour.astype(np.int8)
#     df['dow']=df['dt'].dt.dayofweek.astype(np.int8)
#     df['is_weekend']=(df['dow']>=5).astype(np.int8)
#     df['is_after_hours']=((df['hour']<8)|(df['hour']>18)).astype(np.int8)
#     df['is_night']=((df['hour']<7)|(df['hour']>22)).astype(np.int8)
#     return df

# def add_user_context(df,ctx):
#     cols=['role_id','dept_id','team_id','psy_O','psy_C','psy_E','psy_A','psy_N']
#     return pd.concat([df,pd.DataFrame(df['user'].map(lambda u:ctx.get(u,[0]*8)).tolist(),columns=cols)],axis=1)

# ================= 4. CHUNK PROCESS =================
# ================= 4. CHUNK PROCESS (UPDATED) =================
def process_chunk(chunk, act, ctx, mal_ids):
    # ================= 1. BASE CLEANING =================
    chunk['id'] = chunk['id'].astype(str).str.replace('{','').str.replace('}','')
    chunk['dt'] = pd.to_datetime(chunk['date'], errors='coerce')
    chunk = chunk.dropna(subset=['dt'])

    # ================= 2. TIME FEATURES =================
    chunk['ts'] = (chunk['dt'].astype('int64') // 10**9).astype(np.int64)
    chunk['hour'] = chunk['dt'].dt.hour.astype(np.int8)
    chunk['dow'] = chunk['dt'].dt.dayofweek.astype(np.int8)
    chunk['is_weekend'] = (chunk['dow'] >= 5).astype(np.int8)
    # UEBA Standard: Night (22h-7h), After Hours (18h-8h)
    chunk['is_after_hours'] = ((chunk['hour'] < 8) | (chunk['hour'] > 18)).astype(np.int8)
    chunk['is_night']       = ((chunk['hour'] < 7) | (chunk['hour'] > 22)).astype(np.int8)
    
    # ================= 3. USER CONTEXT =================
    ctx_cols = ['role_id','dept_id','team_id','psy_O','psy_C','psy_E','psy_A','psy_N']
    u_vals = [ctx.get(u, [0]*8) for u in chunk['user']]
    chunk_ctx = pd.DataFrame(u_vals, columns=ctx_cols, index=chunk.index)
    chunk = pd.concat([chunk, chunk_ctx], axis=1)

    # ================= 4. MULTICLASS LABEL =================
    # 0: benign | 1: logon | 2: device | 3: file | 4: email | 5: http
    act_type_code = ACT_MAP[act]
    chunk['label'] = np.where(chunk['id'].isin(mal_ids), act_type_code, 0).astype(np.int8)
    chunk['activity_type'] = act_type_code # Feature cho model biết loại log

    # ================= 5. FEATURE ENGINEERING (LOGIC MỚI) =================
    # Khởi tạo tất cả cột đặc trưng bằng 0
    # Đặt tên cột rõ nghĩa thay vì f1, f2...
    features = [
        # File features
        'file_category', 'file_content_len', 'file_content_words', 
        'file_drive_type', 'file_depth',
        # HTTP features
        'http_category', 'http_url_len', 'http_url_depth', 
        'http_content_len', 'http_content_words',
        # Email features
        'email_recipients_count', 'email_attachment_count', 'email_is_external',
        'email_size', 'email_content_len', 'email_content_words',
        # Session features
        'is_session_start'
    ]
    
    for f in features:
        chunk[f] = 0.0 # Dùng float để tránh lỗi tính toán, sau này model tự xử lý

    # ---------------------------------------------------------
    # A. XỬ LÝ FILE (Logic mapping từ f1 -> f5)
    # ---------------------------------------------------------
    if act == 'file':
        fname = chunk['filename'].fillna('').astype(str)
        content = chunk['content'].fillna('').astype(str)
        fname_lower = fname.str.lower()

        # F1: File Category (1:Other, 2:Archive, 3:Image, 4:Doc, 5:Text, 6:Exe)
        # Mặc định là 1
        chunk['file_category'] = 1 
        chunk.loc[fname_lower.str.contains(r'\.zip|\.rar|\.7z', regex=True), 'file_category'] = 2
        chunk.loc[fname_lower.str.contains(r'\.jpg|\.png|\.bmp', regex=True), 'file_category'] = 3
        chunk.loc[fname_lower.str.contains(r'\.doc|\.pdf|\.xls|\.ppt', regex=True), 'file_category'] = 4
        chunk.loc[fname_lower.str.contains(r'\.txt|\.cfg|\.csv', regex=True), 'file_category'] = 5
        chunk.loc[fname_lower.str.contains(r'\.exe|\.sh|\.bat', regex=True), 'file_category'] = 6

        # F2, F3: Content Length & Word Count
        chunk['file_content_len'] = content.str.len()
        chunk['file_content_words'] = content.str.count(' ') + 1

        # F4: Drive Type (1: C local, 2: R removeable/network, 0: Other)
        chunk['file_drive_type'] = np.where(fname.str.startswith('C'), 1, 
                                   np.where(fname.str.startswith('R'), 2, 0))

        # F5: File Depth (Đếm số dấu backslash)
        chunk['file_depth'] = fname.str.count(r'\\')

    # ---------------------------------------------------------
    # B. XỬ LÝ HTTP (Logic mapping từ f6 -> f10)
    # ---------------------------------------------------------
    elif act == 'http':
        url = chunk['url'].fillna('').astype(str)
        content = chunk['content'].fillna('').astype(str)
        url_lower = url.str.lower()

        # F6: HTTP Category (1:Other, 2:Social, 3:Cloud, 4:Job, 5:Hack/Leak)
        chunk['http_category'] = 1
        chunk.loc[url_lower.str.contains(r'facebook|twitter|instagram', regex=True), 'http_category'] = 2
        chunk.loc[url_lower.str.contains(r'dropbox|drive\.google|onedrive|box', regex=True), 'http_category'] = 3
        chunk.loc[url_lower.str.contains(r'job|indeed|recruit|linkedin', regex=True), 'http_category'] = 4
        chunk.loc[url_lower.str.contains(r'wikileaks|hack|crack|keylog', regex=True), 'http_category'] = 5

        # F7, F8: URL Stats
        chunk['http_url_len'] = url.str.len()
        chunk['http_url_depth'] = url.str.count('/') - 2 # Trừ đi 2 dấu // trong http://
        chunk.loc[chunk['http_url_depth'] < 0, 'http_url_depth'] = 0

        # F9, F10: Content Stats
        chunk['http_content_len'] = content.str.len()
        chunk['http_content_words'] = content.str.count(' ') + 1

    # ---------------------------------------------------------
    # C. XỬ LÝ EMAIL (Logic mapping từ f11 -> f19 & fix External)
    # ---------------------------------------------------------
    elif act == 'email':
        to_s = chunk['to'].fillna('').astype(str)
        bcc_s = chunk['bcc'].fillna('').astype(str) # Có thể null
        content = chunk['content'].fillna('').astype(str)

        # F11: Total Recipients
        n_to = to_s.apply(lambda x: x.count(';') + 1 if x else 0)
        n_bcc = bcc_s.apply(lambda x: x.count(';') + 1 if x else 0)
        chunk['email_recipients_count'] = n_to + n_bcc

        # F12: Attachments Count
        chunk['email_attachment_count'] = chunk['attachments'].fillna(0) # Trong file csv đã có cột attachments đếm sẵn hoặc list

        # F13: External Check (FIX LOGIC)
        # Logic: (To chứa dtaa.com) VÀ (Bcc rỗng HOẶC chứa dtaa.com) -> Nội bộ (0)
        # Ngược lại -> External (1)
        is_internal_to = to_s.str.contains('dtaa.com', regex=False)
        is_internal_bcc = (bcc_s == '') | bcc_s.str.contains('dtaa.com', regex=False)
        
        # Nếu thuần nội bộ thì 0, có dính ngoài thì 1
        chunk['email_is_external'] = np.where(is_internal_to & is_internal_bcc, 0, 1)

        # F17: Size
        chunk['email_size'] = chunk['size'].fillna(0)

        # F18, F19: Content Stats
        chunk['email_content_len'] = content.str.len()
        chunk['email_content_words'] = content.str.count(' ') + 1

    # ---------------------------------------------------------
    # D. XỬ LÝ LOGON / DEVICE (Session Start)
    # ---------------------------------------------------------
    elif act == 'logon':
        chunk['is_session_start'] = (chunk['activity'] == 'Logon').astype(np.int8)
    
    elif act == 'device':
        chunk['is_session_start'] = (chunk['activity'] == 'Connect').astype(np.int8)

    # ================= 6. FINAL SELECT =================
    # Chọn đúng các cột đặc trưng Action-Level (~30 cột)
    final_cols = [
        'user', 'ts', 'label', 'activity_type',
        # Time
        'hour', 'dow', 'is_weekend', 'is_after_hours', 'is_night',
        # Context
        'role_id', 'dept_id', 'team_id', 'psy_O', 'psy_C', 'psy_E', 'psy_A', 'psy_N'
    ] + features # Cộng thêm list features đã define ở trên

    return chunk[final_cols].fillna(0)

# ================= 5. FILE RUN =================
def run_file(act,ctx,mal_ids):
    path=os.path.join(SOURCE_DIR,f'{act}.csv')
    tmp=os.path.join(OUTPUT_DIR,f'tmp_{act}')
    os.makedirs(tmp,exist_ok=True)
    for i,ch in enumerate(pd.read_csv(path,chunksize=CHUNK_SIZE,on_bad_lines='skip')):
        process_chunk(ch,act,ctx,mal_ids).to_parquet(f'{tmp}/p{i}.parquet',index=False)

# ================= MAIN =================
start=time.time()
ctx=load_user_context()
mal_ids=load_malicious_ids()

Parallel(n_jobs=NUM_CORES)(
    delayed(run_file)(a,ctx,mal_ids) for a in ACT_MAP
)

files=[os.path.join(r,f) for r,_,fs in os.walk(OUTPUT_DIR) for f in fs if f.endswith('.parquet')]
df=pd.concat([pd.read_parquet(f) for f in files],ignore_index=True)
df.sort_values(['user','ts'],inplace=True)

out=f'{OUTPUT_DIR}/cert_r4.2_action_multiclass.parquet'
df.to_parquet(out,index=False)
print("✅ DONE:",out)
print(df['label'].value_counts())
print("⏱ Time:",(time.time()-start)/60,"mins")


In [ ]:
# # ================= CERT r4.2 ACTION-LEVEL FINAL PIPELINE =================
# # One-cell | Kaggle-ready | Deep Learning + Explainable

# import os, gc, shutil, time
# import pandas as pd
# import numpy as np
# from joblib import Parallel, delayed

# # ================= CONFIG =================
# SOURCE_DIR = '/kaggle/input/cert-r4-2/archive'
# ANSWERS_DIR = '/kaggle/input/cert42-answer/answers'
# OUTPUT_DIR = '/kaggle/working'
# CHUNK_SIZE = 2_000_000
# NUM_CORES = 4

# ACT_MAP = {'logon':1, 'device':2, 'file':3, 'email':4, 'http':5}
# ORG_DOMAIN = 'dtaa.com'
# EXEC_EXT = ('.exe','.zip','.tar')
# SENSITIVE_PREFIX = 'R'
# RISKY_HTTP = ['job','hack','leak','upload','drive','dropbox','keylog']

# # ================= 1. USER CONTEXT =================
# # ================= 1. USER CONTEXT (Đã tích hợp lưu JSON) =================
# import json # Nhớ import json ở đầu script

# def load_user_context():
#     print("Loading LDAP + Psychometric ...")
#     ldap_dir = os.path.join(SOURCE_DIR, 'LDAP')
#     if not os.path.exists(ldap_dir): ldap_dir = SOURCE_DIR

#     csvs = sorted([os.path.join(ldap_dir,f) for f in os.listdir(ldap_dir) if f.endswith('.csv')])
#     users = {}

#     for f in csvs:
#         try:
#             df = pd.read_csv(f)
#             for _,r in df.iterrows():
#                 users[r['user_id']] = r.to_dict()
#         except: pass

#     df_users = pd.DataFrame.from_dict(users, orient='index')

#     psy = os.path.join(SOURCE_DIR,'psychometric.csv')
#     if os.path.exists(psy):
#         df_psy = pd.read_csv(psy)
#         df_users = df_users.merge(df_psy, on='user_id', how='left')
#     else:
#         for c in ['O','C','E','A','N']: df_users[c] = 0.5

#     def enc(col):
#         # Sắp xếp để đảm bảo thứ tự luôn giống nhau mỗi lần chạy
#         return {v:i+1 for i,v in enumerate(sorted(col.fillna('UNK').unique()))}

#     # 1. Tạo Map
#     role_map = enc(df_users['role'])
#     dept_map = enc(df_users['department'])
#     team_map = enc(df_users['team'])

#     # --- ĐOẠN MỚI THÊM VÀO: LƯU JSON NGAY TẠI ĐÂY ---
#     print("Saving Mapping JSONs...")
#     def save_mapping(mapping, name):
#         # Đảo ngược key-value (Số -> Chữ) để sau này tra cứu
#         # Chuyển key thành string để đảm bảo chuẩn JSON
#         inv = {str(v): k for k,v in mapping.items()} 
#         with open(f'{OUTPUT_DIR}/{name}.json','w') as f:
#             json.dump(inv, f, indent=2)

#     save_mapping(role_map, 'role_map')
#     save_mapping(dept_map, 'dept_map')
#     save_mapping(team_map, 'team_map')
    
#     # Lưu luôn act_map (biến toàn cục) cho đủ bộ
#     with open(f'{OUTPUT_DIR}/act_type_map.json','w') as f:
#         json.dump({str(v):k for k,v in ACT_MAP.items()}, f, indent=2)
#     # ------------------------------------------------

#     ctx = {}
#     for _,r in df_users.iterrows():
#         ctx[r['user_id']] = [
#             role_map.get(r['role'],0),
#             dept_map.get(r['department'],0),
#             team_map.get(r['team'],0),
#             float(r['O']),float(r['C']),float(r['E']),float(r['A']),float(r['N'])
#         ]

#     print(f"-> Loaded {len(ctx)} users & Saved JSONs")
#     return ctx

# # ================= 2. LABELS =================
# def load_labels():
#     print("Loading insider labels ...")
#     path = os.path.join(ANSWERS_DIR,'insiders.csv')
#     if not os.path.exists(path): return {}

#     df = pd.read_csv(path)
#     df = df[df['dataset'].astype(str)=='4.2']
#     mal = {}

#     for _,r in df.iterrows():
#         folder = f"r{r['dataset']}-{r['scenario']}"
#         f = os.path.join(ANSWERS_DIR,folder,r['details'])
#         if os.path.exists(f):
#             with open(f) as fh:
#                 for line in fh:
#                     p = line.strip().split(',')
#                     if len(p)>1:
#                         mal[p[1].replace('{','').replace('}','')] = int(r['scenario'])
#     print(f"-> {len(mal)} malicious actions")
#     return mal

# # ================= 3. FEATURE PARSERS =================
# def time_features(df):
#     df['dt'] = pd.to_datetime(df['date'], errors='coerce')
#     df = df.dropna(subset=['dt'])
#     df['ts'] = (df['dt'].astype('int64')//10**9).astype(np.int64)
#     df['hour'] = df['dt'].dt.hour.astype(np.int8)
#     df['dow'] = df['dt'].dt.dayofweek.astype(np.int8)
#     df['is_weekend'] = df['dow'].isin([5,6]).astype(np.int8)
#     df['is_night'] = ((df['hour']<7)|(df['hour']>18)).astype(np.int8)
#     return df

# def user_context(df, ctx):
#     cols = ['role_id','dept_id','team_id','psy_O','psy_C','psy_E','psy_A','psy_N']
#     data = df['user'].map(lambda u: ctx.get(u,[0]*8)).tolist()
#     return pd.concat([df, pd.DataFrame(data,columns=cols,index=df.index)], axis=1)

# def content_len(df):
#     if 'content' in df.columns:
#         df['content_len'] = df['content'].fillna('').astype(str).str.len().astype(np.int32)
#     else:
#         df['content_len'] = 0
#     return df

# # ================= 4. CHUNK PROCESS =================
# def process_chunk(chunk, act, ctx, mal):
#     chunk['id'] = chunk['id'].astype(str).str.replace('{','').str.replace('}','')
#     chunk['label'] = chunk['id'].map(mal).fillna(0).astype(np.int8)
#     chunk = time_features(chunk)
#     chunk = user_context(chunk, ctx)
#     chunk = content_len(chunk)

#     chunk['act_type'] = ACT_MAP[act]

#     # init
#     chunk[['email_is_external','email_has_attachment','email_num_attachments',
#            'file_is_executable','file_is_sensitive','http_is_risky']] = 0

#     if act=='email':
#         chunk['email_is_external'] = (~chunk['to'].astype(str).str.contains(ORG_DOMAIN)).astype(np.int8)
#         if 'attachments' in chunk.columns:
#             chunk['email_has_attachment'] = chunk['attachments'].notna().astype(np.int8)
#             chunk['email_num_attachments'] = chunk['attachments'].fillna('').astype(str).str.count(';')+1
#             chunk.loc[chunk['attachments'].isna(),'email_num_attachments']=0

#     if act=='file':
#         chunk['file_is_executable'] = chunk['filename'].astype(str).str.lower().str.endswith(EXEC_EXT).astype(np.int8)
#         chunk['file_is_sensitive'] = chunk['filename'].astype(str).str.startswith(SENSITIVE_PREFIX).astype(np.int8)

#     if act=='http':
#         pat='|'.join(RISKY_HTTP)
#         chunk['http_is_risky'] = chunk['url'].astype(str).str.contains(pat,case=False,na=False).astype(np.int8)

#     final_cols = [
#         'user','ts','label','act_type',
#         'hour','dow','is_weekend','is_night',
#         'role_id','dept_id','team_id',
#         'psy_O','psy_C','psy_E','psy_A','psy_N',
#         'content_len',
#         'email_is_external','email_has_attachment','email_num_attachments',
#         'file_is_executable','file_is_sensitive',
#         'http_is_risky'
#     ]
#     return chunk[final_cols]

# # ================= 5. FILE RUNNER =================
# def process_file(act, ctx, mal):
#     path = os.path.join(SOURCE_DIR,f'{act}.csv')
#     if not os.path.exists(path): return
#     print(f"Processing {act} ...")

#     tmp = os.path.join(OUTPUT_DIR,f'tmp_{act}')
#     os.makedirs(tmp,exist_ok=True)
#     i=0

#     for ch in pd.read_csv(path,chunksize=CHUNK_SIZE,on_bad_lines='skip'):
#         df = process_chunk(ch,act,ctx,mal)
#         df.to_parquet(f'{tmp}/p{i}.parquet',index=False)
#         i+=1

#     print(f"-> {act}: {i} chunks")

# # ================= 6. MERGE =================
# def merge_all():
#     files=[]
#     for r,_,fs in os.walk(OUTPUT_DIR):
#         for f in fs:
#             if f.endswith('.parquet') and 'tmp_' in r:
#                 files.append(os.path.join(r,f))

#     print("Merging parquet files ...")
#     df = pd.concat([pd.read_parquet(f) for f in files],ignore_index=True)
#     df.sort_values(['user','ts'],inplace=True)

#     out = os.path.join(OUTPUT_DIR,'cert_r4.2_action_level.parquet')
#     df.to_parquet(out,index=False)

#     print("✅ FINAL DATASET:", out)
#     print("Shape:", df.shape)
#     print("Labels:\n", df['label'].value_counts())

#     for d in os.listdir(OUTPUT_DIR):
#         if d.startswith('tmp_'): shutil.rmtree(os.path.join(OUTPUT_DIR,d))

# # ================= MAIN =================
# start=time.time()
# ctx = load_user_context()
# mal = load_labels()

# acts=['logon','device','file','email','http']
# Parallel(n_jobs=NUM_CORES)(delayed(process_file)(a,ctx,mal) for a in acts)

# merge_all()
# print(f"Total time: {(time.time()-start)/60:.2f} minutes")


In [ ]:
import pyarrow.dataset as ds
import matplotlib.pyplot as plt
from collections import Counter

PARQUET_PATH = "/kaggle/input/cert-r4-2-multiclass/cert_r4.2_action_multiclass.parquet"
BATCH_SIZE = 1_000_000

dataset = ds.dataset(PARQUET_PATH, format="parquet")

LABEL_MAP = {
    0: "Benign",
    1: "Logon",
    2: "Device",
    3: "File",
    4: "Email",
    5: "HTTP"
}

label_counter = Counter()
total_rows = 0

# ✅ API ĐÚNG CHO PYARROW HIỆN TẠI
for batch in dataset.to_batches(columns=["label"], batch_size=BATCH_SIZE):
    labels = batch.column("label").to_pylist()
    total_rows += len(labels)
    label_counter.update(labels)

print("Label distribution (human-readable):")
for k in sorted(label_counter):
    print(f"{LABEL_MAP.get(k, 'Unknown')}: {label_counter[k]}")

# print("✅ Done streaming parquet")
# print("Total rows:", total_rows)
# print("Label distribution:", dict(sorted(label_counter.items())))

# ===== PLOT =====
items = [(l, label_counter[l]) for l in label_counter]
items.sort(key=lambda x: x[1], reverse=True)

labels_num = [i[0] for i in items]
counts = [i[1] for i in items]
labels_txt = [LABEL_MAP.get(l, "Unknown") for l in labels_num]


plt.figure()
bars = plt.bar(labels_txt, counts)
plt.xlabel("Behavior Class")
plt.ylabel("Number of Events")
plt.title("CERT r4.2 – Action-level Behavior Distribution")
# plt.yscale("log")
# plt.ylim(1, 1e8)

# plt.yticks(
#     [1, 1e5, 1e7],
#     [r"$10^0$", r"$10^4$", r"$10^8$"]
# )
plt.xticks(rotation=0)
for bar in bars:
    height = bar.get_height()
    plt.text(
        bar.get_x() + bar.get_width() / 2,
        height,
        f"{height:,}",        # format có dấu phẩy
        ha='center',
        va='bottom',
        fontsize=9
    )
plt.tight_layout()
plt.show()


In [1]:
import pandas as pd
import numpy as np
import os
import gc

# ================= CẤU HÌNH =================
DATA_DIR = "/kaggle/input/cert-r4-2/archive" 
ANSWERS_DIR = "/kaggle/input/cert42-answer/answers/"

print("🚀 BẮT ĐẦU: TẠO NHẬT KÝ HÀNH VI HÀNG NGÀY (DAILY JOURNAL)")

# ================= 1. LOAD GROUND TRUTH (LẤY ID ĐỘC HẠI) =================
print("\n[1/4] Loading Malicious IDs...")
malicious_map = {} 
target_users = set() # Chỉ lấy log của những kẻ nội gián để soi cho kỹ

try:
    # Lấy danh sách Insider
    df_insiders = pd.read_csv(os.path.join(ANSWERS_DIR, 'insiders.csv'))
    if 'dataset' in df_insiders.columns:
        df_insiders = df_insiders[df_insiders['dataset'].astype(str) == '4.2']
    
    target_users = set(df_insiders['user'].astype(str).unique())
    print(f"   -> Tập trung soi {len(target_users)} kẻ nội gián: {target_users}")

    # Lấy ID chi tiết
    for _, row in df_insiders.iterrows():
        scenario = row['scenario']
        detail_file = row['details']
        full_path = os.path.join(ANSWERS_DIR, f"r4.2-{scenario}", detail_file)
        
        if os.path.exists(full_path):
            with open(full_path, 'r') as fh:
                for line in fh:
                    parts = line.strip().split(',')
                    if len(parts) > 1:
                        try:
                            clean_id = parts[1].replace('{','').replace('}','').strip()
                            if clean_id:
                                malicious_map[clean_id] = f"Scenario {scenario}"
                        except: continue
except Exception as e:
    print(f"❌ Lỗi: {e}")

# ================= 2. LOAD LOGS (CHỈ CỦA INSIDERS) =================
print("\n[2/4] Loading Logs for Target Users Only...")
# Chiến thuật: Chỉ load dòng nào thuộc về target_users để tiết kiệm RAM tối đa
# giúp ta có thể load hết chuỗi hành vi của họ mà không cần sample.

log_files = {'logon': 'logon.csv', 'device': 'device.csv', 'file': 'file.csv', 'http': 'http.csv', 'email': 'email.csv'}
daily_logs = []

for activity, filename in log_files.items():
    path = os.path.join(DATA_DIR, filename)
    if os.path.exists(path):
        print(f"   -> Scanning {filename}...")
        for chunk in pd.read_csv(path, chunksize=1_000_000):
            # 1. Lọc User trước
            chunk = chunk[chunk['user'].isin(target_users)].copy()
            if chunk.empty: continue
            
            # 2. Xử lý
            chunk['id'] = chunk['id'].astype(str)
            chunk['ts'] = pd.to_datetime(chunk['date'], format='%m/%d/%Y %H:%M:%S')
            chunk['date_only'] = chunk['ts'].dt.date
            
            # Gán Action Name
            if 'activity' in chunk.columns:
                chunk['action'] = chunk['activity']
            else:
                chunk['action'] = activity.capitalize()
            
            # Check độc hại
            chunk['scenario'] = chunk['id'].map(malicious_map)
            
            # Lưu các cột cần thiết
            daily_logs.append(chunk[['user', 'date_only', 'ts', 'action', 'scenario']])

# Gộp lại
print("   -> Merging data...")
df_full = pd.concat(daily_logs, ignore_index=True)
df_full.sort_values(['user', 'ts'], inplace=True)
del daily_logs
gc.collect()

print(f"📦 Tổng cộng: {len(df_full):,} dòng log của các Insiders.")

# # ... (Giả sử bạn đã có df_full từ bước trước) ...

# print("\n[3/4] Creating Daily Sequences WITH TIME STATS...")

# # Gom nhóm
# grouped = df_full.groupby(['user', 'date_only'])

# journal_data = []

# for (user, date), group in grouped:
#     # 1. Labeling
#     scenarios = group['scenario'].dropna().unique()
#     if len(scenarios) > 0:
#         day_label = list(scenarios)[0]
#         is_malicious = True
#     else:
#         day_label = 'Benign'
#         is_malicious = False
        
#     # 2. Xử lý Thời gian (QUAN TRỌNG)
#     start_time = group['ts'].min()
#     end_time = group['ts'].max()
#     duration = end_time - start_time
    
#     # Format Duration cho đẹp (VD: 00:15:30)
#     total_seconds = int(duration.total_seconds())
#     hours, remainder = divmod(total_seconds, 3600)
#     minutes, seconds = divmod(remainder, 60)
#     if hours > 0:
#         dur_str = f"{hours}h {minutes}m"
#     elif minutes > 0:
#         dur_str = f"{minutes}m {seconds}s"
#     else:
#         dur_str = f"{seconds}s" # Hành động cực nhanh
        
#     # Check "Giờ Hành Chính" (8h - 18h)
#     start_hour = start_time.hour
#     is_night_work = (start_hour < 7) or (start_hour > 19) # Làm đêm/sáng sớm
    
#     # 3. Chuỗi hành vi
#     actions = group['action'].tolist()
#     if len(actions) > 10:
#         seq_str = " -> ".join(actions[:3]) + f" ...({len(actions)-6})... " + " -> ".join(actions[-3:])
#     else:
#         seq_str = " -> ".join(actions)

#     journal_data.append({
#         'User': user,
#         'Date': date,
#         'Time_Range': f"{start_time.strftime('%H:%M')} - {end_time.strftime('%H:%M')}",
#         'Duration': dur_str,
#         'Is_Night': is_night_work,
#         'Label': day_label,
#         'Sequence': seq_str
#     })

# df_journal = pd.DataFrame(journal_data)
# df_journal['Date'] = pd.to_datetime(df_journal['Date'])
# df_journal.sort_values(['User', 'Date'], inplace=True)

# # ================= 4. HIỂN THỊ KẾT QUẢ =================
# print("\n[4/4] 📖 NHẬT KÝ CHI TIẾT (CÓ THỜI GIAN)")
# print("="*110)

# def print_timeline_with_time(user_id):
#     subset = df_journal[df_journal['User'] == user_id]
#     if subset.empty: return

#     print(f"\n👤 USER: {user_id}")
#     # Header
#     print(f"{'Date':<11} | {'Time':<13} | {'Dur':<8} | {'Label':<12} | {'Sequence'}")
#     print("-" * 110)
    
#     for _, row in subset.iterrows():
#         d_str = row['Date'].strftime('%Y-%m-%d')
#         t_range = row['Time_Range']
#         dur = row['Duration']
#         lbl = row['Label']
#         seq = row['Sequence']
        
#         # Icon trang trí
#         status_icon = "🔴" if lbl != 'Benign' else "🟢"
        
#         # Icon cảnh báo thời gian
#         time_icon = ""
#         if row['Is_Night']: time_icon += "🌙" # Làm đêm
#         if "s" in dur and "m" not in dur and row['Is_Night']: time_icon += "⚡" # Làm đêm mà làm quá nhanh -> Rất nguy hiểm
        
#         print(f"{status_icon} {d_str:<9} | {t_range:<13} | {dur:<8} {time_icon:<2} | {lbl:<12} | {seq}")

# # In thử mẫu
# sample_users = list(target_users)[:3]
# for u in sample_users:
#     print_timeline_with_time(u)


# Giả sử bạn đã có df_full từ các bước trước
# Đảm bảo đã sort
df_full.sort_values(['user', 'ts'], inplace=True)

print("🚀 ĐANG TẠO SMART SESSIONS (CẮT KHI NGHỈ > 30 PHÚT)...")

# 1. Tính khoảng cách thời gian giữa các dòng liên tiếp của cùng 1 user
# shift(1) là lấy dòng phía trước để trừ
df_full['time_diff'] = df_full.groupby('user')['ts'].diff().dt.total_seconds() / 60 # Ra phút

# 2. Đánh dấu Session mới
# Session mới bắt đầu khi:
# - Đổi User
# - Đổi Ngày
# - HOẶC: Khoảng cách thời gian > 30 phút (Idle Time)
IDLE_THRESHOLD = 60 # phút

# Tạo cờ đánh dấu điểm bắt đầu session mới
df_full['new_session_flag'] = (
    (df_full['user'] != df_full['user'].shift(1)) | 
    (df_full['date_only'] != df_full['date_only'].shift(1)) | 
    (df_full['time_diff'] > IDLE_THRESHOLD)
)

# Tạo Session ID chạy tăng dần (Cumulative Sum)
df_full['session_id'] = df_full['new_session_flag'].cumsum()

# 3. GOM NHÓM THEO SESSION ID (Thay vì theo Date)
print("   -> Grouping by Smart Session ID...")

sessions = df_full.groupby('session_id').agg({
    'user': 'first',
    'date_only': 'first',
    'ts': ['min', 'max'],     # Lấy giờ bắt đầu, kết thúc
    'scenario': list,      # Lấy danh sách label để check malicious
    'action': list     # Lấy danh sách hành động
}).reset_index()

# Làm phẳng cột (Flatten columns)
sessions.columns = ['session_id', 'user', 'date', 'start_time', 'end_time', 'scenarios', 'actions']

# ================= 4. XỬ LÝ HIỂN THỊ ĐẸP =================
journal_data = []

for _, row in sessions.iterrows():
    # Check Label
    # Lọc bỏ NaN và 'Benign' để tìm xem có cái nào độc hại không
    bad_labels = [s for s in row['scenarios'] if isinstance(s, str) and s != 'Benign']
    
    if bad_labels:
        # Lấy label độc hại đầu tiên tìm thấy
        session_label = bad_labels[0] 
        is_malicious = True
    else:
        session_label = 'Benign'
        is_malicious = False
    
    # Tính thời lượng
    duration = row['end_time'] - row['start_time']
    # Format Duration
    total_seconds = int(duration.total_seconds())
    minutes, seconds = divmod(total_seconds, 60)
    hours, minutes = divmod(minutes, 60)
    if hours > 0: dur_str = f"{hours}h {minutes}m"
    else: dur_str = f"{minutes}m {seconds}s"

    # Tạo chuỗi hiển thị
    act_list = row['actions']
    
    # LOGIC HIỂN THỊ THÔNG MINH:
    # Nếu session ngắn (< 15 hành động) -> In hết
    # Nếu session dài -> Cắt đầu đuôi
    if len(act_list) > 15:
        seq_str = " -> ".join(act_list[:5]) + f" ...({len(act_list)-10})... " + " -> ".join(act_list[-5:])
    else:
        seq_str = " -> ".join(act_list)

    journal_data.append({
        'User': row['user'],
        'Date': row['date'],
        'Time': f"{row['start_time'].strftime('%H:%M')} - {row['end_time'].strftime('%H:%M')}",
        'Duration': dur_str,
        'Label': session_label,
        'Count': len(act_list),
        'Sequence': seq_str
    })

df_journal = pd.DataFrame(journal_data)

# ================= 5. IN RA KẾT QUẢ ĐỂ SO SÁNH =================
print("\n📖 NHẬT KÝ HÀNH VI (CHIA THEO PHIÊN LÀM VIỆC NHỎ)")
print("="*120)
print(f"{'Date':<11} | {'Time Range':<13} | {'Dur':<8} | {'Label':<12} | {'Seq'}")
print("-" * 120)

# Lấy thử 1 user có hành vi độc hại để in
malicious_users = df_journal[df_journal['Label'] != 'Benign']['User'].unique()
if len(malicious_users) > 0:
    target = malicious_users[0]
    subset = df_journal[df_journal['User'] == target]
    
    for _, row in subset.iterrows():
        icon = "🔴" if row['Label'] != 'Benign' else "🟢"
        print(f"{icon} {str(row['Date']):<11} | {row['Time']:<13} | {row['Duration']:<8} | {row['Label']:<12} | {row['Sequence']}")
else:
    print("Không tìm thấy session độc hại nào để demo.")

🚀 BẮT ĐẦU: TẠO NHẬT KÝ HÀNH VI HÀNG NGÀY (DAILY JOURNAL)

[1/4] Loading Malicious IDs...
   -> Tập trung soi 70 kẻ nội gián: {'FSC0601', 'PSF0133', 'AAM0658', 'RMW0542', 'JGT0221', 'BDV0168', 'CCA0046', 'MYD0978', 'GHL0460', 'IJM0776', 'HJB0742', 'IUB0565', 'TNM0961', 'HBO0413', 'CEJ0109', 'CAH0936', 'LCC0819', 'NWT0098', 'JTM0223', 'MCF0600', 'MSO0222', 'FMG0527', 'AJR0932', 'JLM0364', 'RHL0992', 'XHW0498', 'IKR0401', 'VSS0154', 'RAB0589', 'RAR0725', 'HXL0968', 'DIB0285', 'CQW0652', 'MPM0220', 'CSC0217', 'MOS0047', 'RKD0604', 'DRR0162', 'KRL0501', 'AKR0057', 'BSS0369', 'KPC0073', 'EHB0824', 'EHD0584', 'JRG0207', 'LQC0479', 'WDD0366', 'KLH0596', 'MAS0025', 'DCH0843', 'EDB0714', 'BTL0226', 'JJM0203', 'BIH0745', 'MAR0955', 'AAF0535', 'BBS0039', 'GTD0219', 'MDH0580', 'TAP0551', 'JMB0308', 'CCL0068', 'PPF0435', 'PNL0301', 'EGD0132', 'FTM0406', 'RGG0064', 'LJR0523', 'BLS0678', 'ABC0174'}

[2/4] Loading Logs for Target Users Only...
   -> Scanning logon.csv...
   -> Scanning device.csv...
  

In [6]:
import pandas as pd
import numpy as np
import os

# ================= CẤU HÌNH =================
ANSWERS_DIR = "/kaggle/input/cert42-answer/answers/"

print("🚀 CHẠY LẠI: TẠO SMART SESSIONS & GÁN NHÃN THEO THỜI GIAN (TIME-BASED)")

# 1. LOAD KHUNG GIỜ TẤN CÔNG (INSIDERS.CSV)
print("[1] Loading Attack Windows...")
attack_windows = []
try:
    df_insiders = pd.read_csv(os.path.join(ANSWERS_DIR, 'insiders.csv'))
    if 'dataset' in df_insiders.columns:
        df_insiders = df_insiders[df_insiders['dataset'].astype(str) == '4.2']
        
    df_insiders['start'] = pd.to_datetime(df_insiders['start'], format='%m/%d/%Y %H:%M:%S')
    df_insiders['end'] = pd.to_datetime(df_insiders['end'], format='%m/%d/%Y %H:%M:%S')
    
    # Lưu vào list để dễ so sánh
    for _, row in df_insiders.iterrows():
        attack_windows.append({
            'user': row['user'],
            'start': row['start'],
            'end': row['end'],
            'scenario': f"Scenario {row['scenario']}"
        })
    print(f"   -> Đã load {len(attack_windows)} khung giờ tấn công.")
except Exception as e:
    print(f"❌ Lỗi load insiders.csv: {e}")

# 2. TẠO SMART SESSIONS (CẮT KHI NGHỈ > 30 PHÚT)
# Giả sử df_full đã có từ bước trước (đã load và sort)
print("[2] Grouping Smart Sessions (Idle > 30m)...")

df_full.sort_values(['user', 'ts'], inplace=True)
df_full['time_diff'] = df_full.groupby('user')['ts'].diff().dt.total_seconds() / 60 

# Tạo Session ID
df_full['new_session_flag'] = (
    (df_full['user'] != df_full['user'].shift(1)) | 
    (df_full['date_only'] != df_full['date_only'].shift(1)) | 
    (df_full['time_diff'] > 60) # 30 phút nghỉ -> Session mới
)
df_full['session_id'] = df_full['new_session_flag'].cumsum()

# Gom nhóm
sessions = df_full.groupby('session_id').agg({
    'user': 'first',
    'date_only': 'first',
    'ts': ['min', 'max'],
    'action': list
}).reset_index()

# Flatten columns
sessions.columns = ['session_id', 'user', 'date', 'start_time', 'end_time', 'actions']

print(f"   -> Đã tạo được {len(sessions)} phiên làm việc.")

# 3. GÁN NHÃN BẰNG CÁCH SO KHỚP THỜI GIAN
print("[3] Labeling Sessions based on Overlap...")

# Dictionary map tên kịch bản cho đẹp
scenario_descriptions = {
    'Scenario 1': '🔴 Kịch bản 1: Trộm dữ liệu ngoài giờ (Data Exfil)',
    'Scenario 2': '🔴 Kịch bản 2: Trộm sở hữu trí tuệ (IP Theft)',
    'Scenario 3': '🔴 Kịch bản 3: IT Phá hoại (Sabotage)',
    'Scenario 4': '🔴 Kịch bản 4: Gửi tin mật (Email)'
}

journal_data = []

for _, row in sessions.iterrows():
    sess_user = row['user']
    sess_start = row['start_time']
    sess_end = row['end_time']
    
    # Mặc định Benign
    session_label = 'Benign'
    
    # Check xem session này có trùng với khung giờ tấn công nào không
    for atk in attack_windows:
        if atk['user'] == sess_user:
            # Logic giao thoa thời gian (Overlap)
            # (Session Start <= Attack End) AND (Session End >= Attack Start)
            if (sess_start <= atk['end']) and (sess_end >= atk['start']):
                raw_label = atk['scenario']
                session_label = scenario_descriptions.get(raw_label, raw_label)
                break # Tìm thấy 1 cái là đủ
    
    # Tính Duration
    duration = sess_end - sess_start
    total_seconds = int(duration.total_seconds())
    minutes, seconds = divmod(total_seconds, 60)
    hours, minutes = divmod(minutes, 60)
    if hours > 0: dur_str = f"{hours}h {minutes}m"
    else: dur_str = f"{minutes}m {seconds}s"
    
    # Icon thời gian
    start_h = sess_start.hour
    is_night = (start_h < 7) or (start_h > 19)
    
    # Format chuỗi hành động
    act_list = row['actions']
    if len(act_list) > 12:
        seq_str = " -> ".join(act_list[:4]) + f" ...({len(act_list)-8})... " + " -> ".join(act_list[-4:])
    else:
        seq_str = " -> ".join(act_list)

    journal_data.append({
        'User': sess_user,
        'Date': row['date'],
        'Time': f"{sess_start.strftime('%H:%M')} - {sess_end.strftime('%H:%M')}",
        'Duration': dur_str,
        'Is_Night': is_night,
        'Label': session_label,
        'Sequence': seq_str
    })

df_journal = pd.DataFrame(journal_data)

# ================= 4. IN KẾT QUẢ DEMO =================
print("\n📖 NHẬT KÝ HÀNH VI (ĐÃ GÁN NHÃN THÀNH CÔNG)")
print("="*130)
print(f"{'Date':<11} | {'Time Range':<13} | {'Dur':<8} | {'Label (Context)':<50} | {'Seq'}")
print("-" * 130)

# Lấy các session độc hại để in
malicious_sessions = df_journal[df_journal['Label'] != 'Benign']

if not malicious_sessions.empty:
    # Lấy 1 user điển hình có nhiều hành vi độc hại
    target_user = malicious_sessions['User'].value_counts().idxmax()
    
    # Lấy nhật ký của user đó (cả lành tính và độc hại để so sánh)
    subset = df_journal[df_journal['User'] == target_user].sort_values(['Date', 'Time'])
    
    print(f"👤 DEMO USER: {target_user}")
    for _, row in subset.head(10000).iterrows(): # In 15 dòng đầu
        icon = "🔴" if row['Label'] != 'Benign' else "🟢"
        night_icon = "🌙" if row['Is_Night'] else ""
        
        print(f"{icon} {str(row['Date']):<11} | {row['Time']:<13} | {row['Duration']:<8} {night_icon:<1} | {row['Label']:<50} | {row['Sequence']}")
else:
    print("❌ Vẫn chưa tìm thấy session độc hại. Hãy kiểm tra lại file insiders.csv có đúng dataset 4.2 không.")

🚀 CHẠY LẠI: TẠO SMART SESSIONS & GÁN NHÃN THEO THỜI GIAN (TIME-BASED)
[1] Loading Attack Windows...
   -> Đã load 70 khung giờ tấn công.
[2] Grouping Smart Sessions (Idle > 30m)...
   -> Đã tạo được 38240 phiên làm việc.
[3] Labeling Sessions based on Overlap...

📖 NHẬT KÝ HÀNH VI (ĐÃ GÁN NHÃN THÀNH CÔNG)
Date        | Time Range    | Dur      | Label (Context)                                    | Seq
----------------------------------------------------------------------------------------------------------------------------------
👤 DEMO USER: MDH0580
🟢 2010-01-04  | 08:28 - 08:49 | 21m 38s    | Benign                                             | Logon -> Http
🟢 2010-01-04  | 09:54 - 10:11 | 16m 54s    | Benign                                             | Http -> Http
🟢 2010-01-04  | 11:42 - 12:13 | 31m 2s     | Benign                                             | Http -> Http
🟢 2010-01-04  | 15:00 - 15:00 | 0m 0s      | Benign                                             | Http
🟢 2010